## Assignment 3

1.复习课上内容及代码。

2.回答以下理论题目：

2.1 动态规划(DP)可以处理的问题有什么特点 ？

答：

    1. 重复子问题：在分解原问题为子问题，求解子问题的过程中，存在许多重复的计算；
    2. 最优子结构：原问题的最优解包含其子问题的最优解，即原问题的最优解可以由子问题的最优解组合得到；
    3. 无后效性：在当前子问题的决策过程中，不需要考虑前面的问题（上一个原问题）是如何解决的，即状态转移的过程中，当前状态只与前一个状态有关。

2.2 简单描述动态规划的几个步骤？

答：

    1. 根据题目写出状态转移方程，注意不同的状态转移到下一状态的情况；
    2. 状态初始化，重要的是边界的初始化；
    3. 根据状态转移方程，写出代码；
    3. 根据题目找到最优解的情况，返回对应值。

2.3 编辑距离算法的迭代公式是什么？

答：

    1. 边界情况为，两个字符串中任一个长度为0，则函数直接返回另一个非0长度字符串的长度；
    2. 状态转移方程：D[i][j] = min(D[i-1][j] + 1, D[i][j-1] + 1, D[i-1][j-1] + 1 if word1[i]==word2[j] else 0);
    3. 由于D[i][j]表示word1前i个字符与word2前j个字符之间的编辑距离，所以最优解为D[len(word1)][len(word2)]

3.实践题

3.1 完成编辑距离解码函数

# 编辑距离

In [1]:
solution = {}

In [2]:
list_parse_results = [] # 清空 parse 的结果

In [3]:
from functools import lru_cache

In [4]:
@lru_cache(maxsize=2**10)
def edit_distance(string1, string2):
    
    if len(string1) == 0: 
        cost = len(string2)
        while len(string2) > 0:
            solution[("",string2)] = 'ADD {}'.format(string2[-1])
            string2 = string2[:-1]
        return cost #词1是空白 那么距离为词2的长度
    if len(string2) == 0: 
        cost = len(string1)
        while len(string2) > 0:
            solution[("",string2)] = 'DEL {}'.format(string1[-1])
            string1 = string1[:-1]
        return cost
    
    tail_s1 = string1[-1]
    tail_s2 = string2[-1]
    # (distance, operation)
    candidates = [     # D[i-1,j]+1     删除操作 # ABCDECG=>ABCDEG=>ABCDE=>ABCD=>ABC=>AB=>A
        (edit_distance(string1[:-1], string2) + 1, 'DEL {}'.format(tail_s1)),   
        # string 1 delete tail
                       #D[i,j-1]+1  插入操作 # 
        (edit_distance(string1, string2[:-1]) + 1, 'ADD {}'.format(tail_s2)),  
        # string 1 add tail of string2
    ]
    
    if tail_s1 == tail_s2:       #D[i-1,j-1]+0   D[i-1,j-1]=D[bc,bce] ==> D[i,j]=D[bcl,bcek]
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 0, '')
    else:                       #D[i-1,j-1]+1
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 1, 'SUB {} => {}'.format(tail_s1, tail_s2))

    candidates.append(both_forward)  
    
    min_distance, operation = min(candidates, key=lambda x: x[0]) #min（D[i-1,j]+1, D[i,j-1]+1,D[i-1,j-1]+1/0）
    
    solution[(string1, string2)] = operation  #记录在对应的字符串对里该使用的操作

    return min_distance

## 编辑距离最小的两个字符串，互相转换的操作

In [5]:
'''
    解析需要哪些操作的过程，相当于把状态转移的思想反过来，查看在word1与word2的状态时需要哪些操作
'''
def parse_results(word1, word2):
    # 如果 word1 与 word2 不存在，返回
    if not word1 and not word2: return list_parse_results
    # 遇到相同的结尾字符，删除，无需操作
    while word1 and word2 and word1[-1] == word2[-1]:
        word1 = word1[:-1]
        word2 = word2[:-1]
    if not word1 and not word2: return list_parse_results
    # 打印待处理的 word1 与 word2
    print("{} ==> {}".format(word1, word2), end="")
    temp = ' '.join([word1, '==>', word2, ': '])
    # 得到能把word1变为word2的 操作
    opera_letter = solution[(word1, word2)].split(" ")
    # 判断操作符，DEL：删除word1的最后一个词；ADD：在word1后面增加一个词；SUB：将word1的最后一个词替换
    if opera_letter[0] == 'DEL': word1 = word1[:-1]
    elif opera_letter[0] == 'ADD': word1 += opera_letter[1]
    else: word1 = word1[:-1] + opera_letter[-1]
        
    temp += ' '.join(opera_letter)
    # 将操作 添加到 列表中    
    list_parse_results.append(temp)
    # 打印操作
    print(", need to {}".format(list_parse_results[-1]))
    return parse_results(word1, word2)

In [6]:
word1 = 'negn'
word2 = 'neng'

In [7]:
print(edit_distance(word1,word2))
print(solution)

2
{('', 'neng'): 'ADD g', ('', 'nen'): 'ADD n', ('', 'ne'): 'ADD e', ('', 'n'): 'ADD n', ('n', 'n'): '', ('n', 'ne'): 'ADD e', ('n', 'nen'): 'ADD n', ('n', 'neng'): 'ADD g', ('ne', 'n'): 'DEL e', ('ne', 'ne'): '', ('ne', 'nen'): 'ADD n', ('ne', 'neng'): 'ADD g', ('neg', 'n'): 'DEL g', ('neg', 'ne'): 'DEL g', ('neg', 'nen'): 'SUB g => n', ('neg', 'neng'): '', ('negn', 'n'): 'DEL n', ('negn', 'ne'): 'DEL n', ('negn', 'nen'): '', ('negn', 'neng'): 'DEL n'}


In [8]:
list_parse_results = []
parse_results(word1,word2)

negn ==> neng, need to negn ==> neng : DEL n
ne ==> nen, need to ne ==> nen : ADD n


['negn ==> neng : DEL n', 'ne ==> nen : ADD n']

3.2 完成拼音自动纠错代码

如何在不带空格的时候完成自动修整？--> 如何完成拼音的自动分割？   
###### 提示：使用第一节课提到的语言模型!

# 语料预处理

In [9]:
# 语料预处理1 —— 语料导入
chinese_dataset = '../article_9k.txt'
CHINESE_CHARATERS = open(chinese_dataset).read()
print(CHINESE_CHARATERS[:100])

此外自本周6月12日起除小米手机6等15款机型外其余机型已暂停更新发布含开发版体验版内测稳定版暂不受影响以确保工程师可以集中全部精力进行系统优化工作有人猜测这也是将精力主要用到MIUI9的研发之中MI


In [10]:
# 语料预处理2 —— 文字转拼音
import pinyin
print(pinyin.get('你好，中国',format='strip',delimiter=' ')) # 举例
def chinese_to_pinyin(character): # 定义文字转拼音的函数
    return pinyin.get(character, format='strip', delimiter=' ')

ni hao ， zhong guo


In [11]:
CHINESE_CHARATERS_COPYS = chinese_to_pinyin(CHINESE_CHARATERS) # 将文字语料 转为 拼音
print(len(CHINESE_CHARATERS_COPYS))

129433034


In [12]:
import re
def tokens(text): # 将拼音全部变为小写 匹配所有拼音
    "List all the pinyin characters"
    return re.findall('[a-z]+',text.lower())

In [13]:
from collections import Counter, defaultdict

PINYIN_COUNT = Counter(tokens(CHINESE_CHARATERS_COPYS))
# ?PINYIN_COUNT

In [14]:
PINYIN_COUNT.most_common(20) # 存在 单字母拼音 如何去掉

[('shi', 860634),
 ('de', 809887),
 ('n', 691114),
 ('yi', 682478),
 ('ji', 645276),
 ('guo', 430042),
 ('zhong', 409418),
 ('zhi', 398612),
 ('xin', 359619),
 ('li', 355441),
 ('zai', 334106),
 ('wei', 326301),
 ('hua', 304941),
 ('yu', 302949),
 ('she', 293312),
 ('he', 285689),
 ('bu', 281533),
 ('ri', 278379),
 ('jin', 278141),
 ('you', 277726)]

In [15]:
need_to_del = []
for k,v in PINYIN_COUNT.items():
    if len(k) == 1: need_to_del.append(k)
for pinyin in need_to_del:
    del PINYIN_COUNT[pinyin]

In [16]:
PINYIN_COUNT.most_common(20) # 不存在单字母拼音

[('shi', 860634),
 ('de', 809887),
 ('yi', 682478),
 ('ji', 645276),
 ('guo', 430042),
 ('zhong', 409418),
 ('zhi', 398612),
 ('xin', 359619),
 ('li', 355441),
 ('zai', 334106),
 ('wei', 326301),
 ('hua', 304941),
 ('yu', 302949),
 ('she', 293312),
 ('he', 285689),
 ('bu', 281533),
 ('ri', 278379),
 ('jin', 278141),
 ('you', 277726),
 ('xian', 269047)]

In [32]:
TOKEN = tokens(CHINESE_CHARATERS_COPYS)

TOKEN_BIGRAM = [''.join(TOKEN[i:i+2]) for i in range(len(TOKEN[:-2]))]
words_count_bigram = Counter(TOKEN_BIGRAM)
print(words_count_bigram.most_common(20))

[('yueri', 165274), ('xinhua', 151823), ('huashe', 145795), ('nn', 136525), ('nxin', 89783), ('zhongguo', 87864), ('waidai', 83330), ('nianyue', 77695), ('jizhe', 65213), ('nwai', 62999), ('erxian', 62428), ('daier', 61783), ('rin', 54162), ('zhaopian', 52347), ('nyue', 51736), ('shezhao', 50584), ('guoji', 47659), ('sheji', 45223), ('rizai', 44410), ('fazhan', 41026)]


In [17]:
import time
from functools import wraps 
def time_consuming(func):  #告诉我们一个函数使用的时间
    @wraps(func)
    def wrapper(*args):
        start = time.time()
        t = func(*args)  #调用传入的func
        end = time.time()
        print('used time : {}'.format(end-start)) #func这个函数花费的时间
        return t
    return wrapper

In [18]:
def known(words):
    'Return the pinyin in our data'
    return [w for w in words if w in PINYIN_COUNT]

def edits0(word):
    'Return all strings that are zero edits away from word (i.e., just word itself).'
    return [word]

alphabet = 'abcdefghijklmnopqrstuvwxyz'

def splits(word):
    'Return a list of all possible (first, rest) pairs that comprise pinyin.'
    return [(word[:i], word[i:]) for i in range(len(word)+1)]

def edits1(word):
    'Return all strings that are one edit away from this pinyin.'
    pairs = splits(word)   
    # =====>>> 更改源码为 ''.join 避免重复赋值
    deletes = [''.join([a,b[1:]]) for (a,b) in pairs if b] # input pinyin -删除操作 去掉b[0]
    replaces = [''.join([a,c,b[1:]]) for (a,b) in pairs for c in alphabet if b] #替换  b[0]替换为c
    inserts = [''.join([a,c,b]) for (a,b) in pairs for c in alphabet] #插入操作 在a和b之间插入字母
    return list(set(deletes + replaces + inserts))

def edits2(word):
    'Return all strings that are two edits away from this pinyin.'
    return {e2 for e1 in edits1(word) for e2 in edits1(e1)} #要计算编辑距离为2的词 可以通过计算与原词编辑距离为1的词的编辑距离为1的词

@time_consuming
def get_the_similar_word(word, words): # 这个函数 根据 对位字符或非对位字符的不同对应方式 给予每个编辑距离相近的更细致的评价
    nums_count = [0 for _ in range(len(words))]
    for i, waited_words in enumerate(words):
        for j, w in enumerate(waited_words):
            if w in word:
                if j < len(word):
                    if w == word[j]: nums_count[i] += len(word)-j+1
                    else: nums_count[i] += (len(word)-j+1) / 2
                else: nums_count[i] += 0.1
            else: nums_count[i] -= 0.1
    
    return sorted(dict(zip(words, nums_count)).items(), key=lambda x:(x[1],x[0]), reverse=True)[0][0]

def correct(word): #pinyin candidates piyin pingyin aogyin .... 
    'Find the most possible pinyin based on edit distance'
    # Prefer edit distance 0, then 1, then 2; otherwist default to word itself
    is_the_word_right = known(edits0(word)) # 编辑距离为0 那就是本身
    if is_the_word_right: return word # 如果编辑距离（本身）在词表中，则直接返回
    else: candidates = list(set(known(edits1(word))+ known(edits2(word))))  #计算所有与输入词编辑距离为0，1，2的词
    #print(candidates)
    
    # 得到 所有可能的拼音 与其在拼音表中对应的 概率
    prob_candidates = dict(zip(candidates, [PINYIN_COUNT.get(p,0) for p in candidates]))
    #print(prob_candidates)
    
    # 按照 拼音表中的概率 排序
    prob_candidates = sorted(prob_candidates.items(), key=lambda x:(x[1],x[0]), reverse=True)
    #print(prob_candidates[:10])
    
    # 其它规则，我这里写的是： 先选出待选词中的前一半词，把这些词与 输入的word 进行匹配，输出匹配得最多的待选word
    # 大概意思就是：negn 应该与 neng 更像，因为它们的长度更相似
    return get_the_similar_word(word,[word for word, prob in prob_candidates[:len(prob_candidates)//2]]) #取出在所有candidate里在语料库里出现次数最多的词

In [19]:
correct('negn')

used time : 3.4332275390625e-05


'neng'

In [20]:
correct('bii')

used time : 0.00010347366333007812


'bi'

In [21]:
correct('jiaa')

used time : 4.410743713378906e-05


'jia'

In [23]:
correct('zhng')

used time : 9.107589721679688e-05


'zhong'

In [22]:
correct('qii')

used time : 7.43865966796875e-05


'qi'

In [35]:
from functools import lru_cache

In [44]:
def prob_1(word):
    if word in PINYIN_COUNT: return PINYIN_COUNT(word) / len(PINYIN_COUNT)
    else: return 1 / len(PINYIN_COUNT)

def prob_2(word1, word2):  # p(w1,w2) = count(w1,2)/count(w1)
    if word1 + word2 in words_count_bigram: return words_count_bigram[word1+word2] / PINYIN_COUNT[word1] + 1
    else:
        return 1 / len(words_count_bigram)

In [101]:
'''
    dp[i][j] ==> sentences 中 下标从i到j的拼音 存在的 概率
'''
@time_consuming
def get_splited_PinYin(sentences):
    n = len(sentences)
    
    dp = {} # 用于记录 所有可能的 字符串：（起始下标，终止下标） 与 其对应的概率
    for i in range(n):
        for j in range(i+1, n):
            prob = PINYIN_COUNT.get(sentences[i:j+1],0)
            if prob: dp[(i,j)] = prob
    
    indexes, probs = list(dp.keys())[::-1], list(dp.values())[::-1]
    
    print('Indexes: {}'.format(indexes))
#     print('Probs: {}'.format(probs))
    
    res = []
    
    # 此函数 用于 找出下标跨度最大的 组合
    def find_the_most_possible_one(start_ind, index):
        if start_ind == 0: return
        light, ind, max_distance = 0, -1, 0
        for i, (left_ind, right_ind) in enumerate(indexes):
            if right_ind+1 == start_ind:
                if right_ind - left_ind + 1 > max_distance:
                    ind = i
                    max_distance = right_ind - left_ind + 1
        if ind == -1: 
            res.append(sentences[indexes[index+1][0]:start_ind])
            find_the_most_possible_one(indexes[index+1][0], index+1)
        else: 
            res.append(sentences[indexes[ind][0]:indexes[ind][1]+1])
            find_the_most_possible_one(indexes[ind][0], ind)

    # 要从最后一个拼音开始找
    find_the_most_possible_one(n, 0)
    #print('After: {}'.format(res))

    res = ' '.join(res[::-1])
    print('Bofre the correction: {}'.format(res))
    return res

In [92]:
get_splited_PinYin('wonegnxihuannima')

Indexes: [(14, 15), (12, 13), (10, 11), (8, 11), (8, 10), (8, 9), (6, 7), (2, 3), (0, 1)]
Bofre the correction: wo negn xi huan ni ma
used time : 0.0003962516784667969


'wo negn xi huan ni ma'

In [26]:
def correct_sequence_pinyin(text_pinyin):
    return ' '.join(map(correct, get_splited_PinYin(text_pinyin).split()))

In [93]:
correct_sequence_pinyin('wonegnxihuannima')

Indexes: [(14, 15), (12, 13), (10, 11), (8, 11), (8, 10), (8, 9), (6, 7), (2, 3), (0, 1)]
Bofre the correction: wo negn xi huan ni ma
used time : 0.0006122589111328125
used time : 3.695487976074219e-05


'wo neng xi huan ni ma'

In [94]:
correct_sequence_pinyin('woxiangshangqinghuaxdaxue')

Indexes: [(22, 24), (22, 23), (20, 21), (16, 18), (16, 17), (12, 15), (12, 14), (12, 13), (9, 11), (9, 10), (8, 11), (8, 10), (8, 9), (7, 11), (7, 10), (7, 9), (4, 6), (4, 5), (2, 6), (2, 5), (2, 4), (2, 3), (0, 1)]
Bofre the correction: wo xiang shang qing huax da xue
used time : 0.0007498264312744141
used time : 5.4836273193359375e-05


'wo xiang shang qing hua da xue'

In [103]:
correct_sequence_pinyin('nizhangdezhanhaokan')

Indexes: [(17, 18), (16, 18), (16, 17), (14, 15), (13, 15), (13, 14), (11, 12), (10, 12), (10, 11), (9, 12), (9, 11), (7, 8), (4, 6), (4, 5), (3, 6), (3, 5), (3, 4), (2, 6), (2, 5), (2, 4), (0, 1)]
Bofre the correction: ni zhang de zhan hao kan
used time : 0.0003974437713623047


'ni zhang de zhan hao kan'